In [5]:
# Import necessary libralies
import pickle
import numpy as np
import os
import mne
import pandas as pd
import seaborn as sns
pickle.format_version

'4.0'

## 1.ETL

In [6]:
import os
# main_path = os.getcwd()
# file_name = os.listdir(main_path + '/Clean_signal/')

In [9]:
# Rename all the files to exp with number 
import os
main_path = os.getcwd()
main_path
all_file = os.listdir(path='./Clean_signal/')
all_file
file_name = list()
index_counter = 0
for file in all_file:
    if file[0:5] == 'clean':
        file_name.append(file)

In [10]:
# Check filename
print(file_name)

['clean_exp01.csv', 'clean_exp02.csv', 'clean_exp03.csv', 'clean_exp04.csv', 'clean_exp05.csv', 'clean_exp06.csv', 'clean_exp07.csv', 'clean_exp08.csv', 'clean_exp09.csv', 'clean_exp10.csv', 'clean_exp11.csv', 'clean_exp12.csv', 'clean_exp13.csv', 'clean_exp14.csv', 'clean_exp15.csv', 'clean_exp16.csv', 'clean_exp17.csv']


In [43]:
# file_name = ['clean_exp01.csv', 'clean_exp02.csv', 'clean_exp03.csv',
#  'clean_exp04.csv', 'clean_exp06.csv', 'clean_exp07.csv',
#  'clean_exp08.csv', 'clean_exp09.csv', 'clean_exp10.csv',
#  'clean_exp11.csv', 'clean_exp12.csv', 'clean_exp13.csv',
#  'clean_exp14.csv', 'clean_exp15.csv', 'clean_exp16.csv']

In [11]:
all_file_path = main_path + '/Clean_signal/'
X_list = list()
for file in file_name:
    file_path = all_file_path + file
    df = pd.read_csv(file_path, index_col=None)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    X_list.append(df)    

In [12]:
df.head()

,Fp1,Fp2,F7,F3,F4,F8,T3,C3,C4,T4,T5,P3,P4,T6,O1,O2
0,1.558541e-13,1.202787e-12,-1.822815e-12,-6.666149e-13,-1.524659e-14,-9.656176e-14,1.778769e-13,-7.318365e-13,-3.218725e-14,6.776264e-15,2.463172e-12,-5.293956e-13,1.679243e-13,2.774456e-12,-3.303428e-13,2.217109e-13
1,-2.495299e+02,-2.591266e+02,-2.392414e+02,-2.472481e+02,-1.342918e+02,-2.469675e+02,-2.368676e+02,-2.375381e+02,8.964020e+01,9.114929e+01,9.105020e+01,7.684951e+01,1.383037e+02,-4.155527e+02,-3.083336e+02,-1.145710e+02
2,-1.022146e+02,-1.071695e+02,-8.913196e+01,-9.320015e+01,-7.188064e+01,-9.669399e+01,-7.360927e+01,-7.934479e+01,-8.483305e+00,-3.966042e+00,6.921319e-02,-5.860820e+00,6.340924e+00,-1.006591e+02,-7.385641e+01,-3.186257e+01
3,-1.894707e+02,-1.985657e+02,-1.666088e+02,-1.773347e+02,-1.206077e+02,-1.861381e+02,-1.566157e+02,-1.616160e+02,1.053098e+01,1.359837e+01,2.570865e+01,1.375228e+01,4.319836e+01,-2.383818e+02,-1.751361e+02,-7.472951e+01
4,-1.976600e+02,-2.088452e+02,-1.691085e+02,-1.829386e+02,-1.219805e+02,-1.981792e+02,-1.664133e+02,-1.711082e+02,5.430791e+00,5.540176e+00,2.039321e+01,9.522058e+00,4.153111e+01,-2.713544e+02,-1.985949e+02,-9.082392e+01


In [13]:
X = np.array(X_list)
X.shape

(17, 45000, 16)

In [14]:
# Reshape
X = np.transpose(X, (0, 2, 1))
X.shape

(17, 16, 45000)

In [16]:
y_path = './label_exp.csv'
df = pd.read_csv(y_path)
y = np.array(df['label'])
y.shape

(17,)

In [17]:
import torch   #general pytorch
import torch.nn as nn  #neural network module
import torch.nn.functional as F  #useful functions like softmax, or relu

#pip install torchvision; conda install torchvision
from torchvision import datasets, transforms  #transforms for image processing
from torch.utils.data import DataLoader, Dataset       #dataloader for preparing batch

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable

In [18]:
# Create dataset
tensor_X = torch.tensor(X)
tensor_y = torch.tensor(y)
dataset = TensorDataset(tensor_X, tensor_y)

In [19]:
for x, y in dataset:
    print(x.shape, y.shape)
    break

torch.Size([16, 45000]) torch.Size([])


## 2. EDA - Spectral Analysis

In [20]:
# Meta data
ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
ch_types = ['eeg'] * len(ch_names)
sfreq = 125
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

# Create epochs
epochs = mne.EpochsArray(tensor_X, info)

# Set headset mapping
epochs.set_montage('standard_1020')
print('------------------')
print(f'{epochs._data.shape=}')
print('------------------')
epochs

Not setting metadata
17 matching events found
No baseline correction applied
0 projection items activated
------------------
epochs._data.shape=(17, 16, 45000)
------------------


Number of events,17
Events,1: 17
Time range,0.000 – 359.992 sec
Baseline,off


In [21]:
# Compute power spectrum density

epoch = epochs.__getitem__(0)
spectrum = epoch.compute_psd(method="welch", n_fft=125) #get Spectrum object from epochs
psds, freqs = spectrum.get_data(return_freqs=True)
print(psds.shape, freqs.shape)

Effective window size : 1.000 (s)
(1, 16, 63) (63,)


In [22]:
# Add log for vissiblity
spectrum = epochs.compute_psd(method='welch', n_fft=125)
psds, freqs = spectrum.get_data(return_freqs=True)
psds = 10 * np.log10(psds)
print(psds.shape, freqs.shape)

Effective window size : 1.000 (s)
(17, 16, 63) (63,)


In [23]:
# 1. First step, find the mean of all channels
psds = psds.mean(axis=1)
psds.shape

(17, 63)

In [24]:
#2. Seperate to the range of bands
# delta 0-3 Hz 
delta = psds[:, :4]
# theta 4-7 Hz
theta = psds[:, 4:8]
# alpha 8-12 Hz
alpha = psds[:, 8:13]
# beta 13-30 Hz
beta = psds[:, 13:31]
# gamma 25-43 Hz
gamma = psds[:, 25:44]
# slow 4-13 Hz
slow = psds[:, 4:14]
# low beta 13-17 Hz
low_beta = psds[:, 13:18]

In [25]:
print(f'delta shape is {delta.shape}')
print(f'theta shape is {theta.shape}')
print(f'alpha shape is {alpha.shape}')
print(f'beta shape is {beta.shape}')
print(f'gamma shape is {gamma.shape}')
print(f'slow shape is {slow.shape}')
print(f'low_beta shape is {low_beta.shape}')

delta shape is (17, 4)
theta shape is (17, 4)
alpha shape is (17, 5)
beta shape is (17, 18)
gamma shape is (17, 19)
slow shape is (17, 10)
low_beta shape is (17, 5)


In [26]:
delta = delta.mean(axis=1).reshape(-1, 1)
theta = theta.mean(axis=1).reshape(-1, 1)
alpha = alpha.mean(axis=1).reshape(-1, 1)
beta = beta.mean(axis=1).reshape(-1, 1)
gamma = gamma.mean(axis=1).reshape(-1, 1)
slow = slow.mean(axis=1).reshape(-1, 1)
low_beta = low_beta.mean(axis=1).reshape(-1, 1)

In [27]:
print(f'delta shape is {delta.shape}')
print(f'theta shape is {theta.shape}')
print(f'alpha shape is {alpha.shape}')
print(f'beta shape is {beta.shape}')
print(f'gamma shape is {gamma.shape}')
print(f'slow shape is {slow.shape}')
print(f'low_beta shape is {low_beta.shape}')

delta shape is (17, 1)
theta shape is (17, 1)
alpha shape is (17, 1)
beta shape is (17, 1)
gamma shape is (17, 1)
slow shape is (17, 1)
low_beta shape is (17, 1)


In [28]:
# Stack them together for ploting
_temp = [delta, theta, alpha, beta, gamma, slow, low_beta]
all_bands = np.hstack(_temp)
all_bands.shape

(17, 7)

In [29]:
# Turn all band into dataframe
df_psd = pd.DataFrame(all_bands, columns= ['delta', 'theta', 'alpha', 'beta', 'gamma', 'slow', 'low_beta'])
df_psd.head()

,delta,theta,alpha,beta,gamma,slow,low_beta
0,5.816752,-0.211333,-4.062054,-8.697476,-12.212243,-2.739044,-7.487652
1,12.457123,6.573959,2.867248,-1.010743,-2.015134,4.173378,0.016220
2,7.680673,1.548254,6.900073,-4.632303,-9.638861,4.243176,-2.389733
3,9.857033,7.392447,6.185955,0.347076,-0.899249,6.241433,1.088514
4,20.362204,14.998787,8.750110,3.305604,-0.751017,11.077218,5.997419


#### Asymmetry

In [30]:
# Chaky code
def df_to_raw(df, convert=False):
    ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
    ch_types = ['eeg'] * len(ch_names)
    df = df.T

    #df[:] *= 1e-6 # mne assume volt data # We already converted in the ETA process

    sampling_rate = 125 #Hz
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sampling_rate)
    raw = mne.io.RawArray(df, info)
    raw.set_montage('standard_1020')
    # plotting, is not work
    # raw.compute_psd().plot()
    return raw

In [31]:
# Put data into mne
main_path = os.getcwd()
eeg = dict()

for index, file in enumerate(file_name):
    clean_signal_path = main_path + '/Clean_signal/' + file
    df = pd.read_csv(clean_signal_path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    raw = df_to_raw(df)
    eeg['exp' + str(index)] = raw
    print(f'{file} | time: {len(df)/125}')


Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp01.csv | time: 360.0
Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp02.csv | time: 360.0
Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp03.csv | time: 360.0
Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp04.csv | time: 360.0
Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp05.csv | time: 360.0
Creating RawArray with float64 data, n_channels=16, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   359.992 secs
Ready.
clean_exp06.csv | time: 360.0
Creating RawArray with float64 data, n_channel

In [32]:
# Get freqeuency
def get_freq(info):
    raw = info
    power, freq = mne.time_frequency.psd_welch(raw,n_fft=125, verbose=True)
    return freq
    
for name, info in eeg.items():
    freq = get_freq(info)
    print(freq)
    break

<decorator-gen-201>:4: FutureWarning: NOTE: psd_welch() is a deprecated function. Function ``psd_welch()`` is deprecated; for Raw/Epochs/Evoked instances use ``spectrum = instance.compute_psd(method="welch")`` instead, followed by ``spectrum.get_data(return_freqs=True)``.


Effective window size : 1.000 (s)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62.]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [33]:
band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta'])
filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]
# band_names = np.array(['Delta', 'Theta', 'Alpha', 'Slow', 'Low_beta'])
# filter_list = [[1,3],[4,7],[8,12], [4,13], [13,17]]
bands = []
for filt in filter_list:
    pt = np.argwhere((freq >= filt[0]) & (freq <= filt[1])).reshape(-1)
    bands.append(pt)
bands = np.array(bands)
print(bands)

[array([1, 2, 3], dtype=int64) array([4, 5, 6, 7], dtype=int64)
 array([ 8,  9, 10, 11, 12], dtype=int64)
 array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30], dtype=int64)
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43],
       dtype=int64)
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64)
 array([13, 14, 15, 16, 17], dtype=int64)]


C:\Users\Guntsv\AppData\Local\Temp\ipykernel_15632\3271316881.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bands = np.array(bands)


In [34]:
#Asymmetry 
features = None
for name, info in eeg.items():
    raw = info
    feature = None
    slow, gamma = None, None
    a_f3, a_f4 = None, None
    a_t7, a_t8 = None, None
    b_f3, b_f4 = None, None
    b_t7, b_t8 = None, None
    #epochs = mne.Epochs(raw, np.array([[125*60*1, 0, 1]]), tmin=0, tmax=30, baseline=(0,30), verbose=False)
    epochs = raw

    print(f'name: {name}')


    for index, band in enumerate(bands):
        
        power,freq = mne.time_frequency.psd_welch(epochs,n_fft=125, verbose=False)
        power = power.squeeze()
        power = 10 * np.log10(power)
        data = power[::,band].mean(axis=1).reshape(1,-1)
        # for asym
        if(band_names[index] == 'Alpha'):
            a_f3 = data[:,raw.ch_names.index('F3')]
            a_f4 = data[:,raw.ch_names.index('F4')]
            # We use t3 as t7 and t4 as t8
            a_t7 = data[:,raw.ch_names.index('T3')]
            a_t8 = data[:,raw.ch_names.index('T4')]
        if(band_names[index] == 'Beta'):
            b_f3 = data[:,raw.ch_names.index('F3')]
            b_f4 = data[:,raw.ch_names.index('F4')]
            # We use t3 as t7 and t4 as t8
            b_t7 = data[:,raw.ch_names.index('T3')]
            b_t8 = data[:,raw.ch_names.index('T4')]

        ####### Mean for visualization #######
        data = data.mean().reshape(1,-1)
        # for relative gamma
        if(band_names[index] == 'Slow'): 
            slow = data
        if(band_names[index] == 'Gamma'): 
            gamma = data

        if(type(feature) == type(None)): 
            print(f'type feature == None band name = {band_names[index]} value = {data}')
            feature = data
        else:
            print(f'band name = {band_names[index]} value = {data}')
            feature = np.concatenate([feature, data], axis=1)
    # print(feature.shape)
    # the eighth feature: relative gamma is slow/gamma
    relative_gamma = slow/gamma
    feature = np.concatenate([feature, relative_gamma], axis=1)

    print(f'relative gamma {relative_gamma}')

    # The asymetry
    alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
    feature = np.concatenate([feature, alpha_frontal], axis=1)

    print(f'alpha_frontal {alpha_frontal}')

    # alpha_temporal
    alpha_temporal = ((a_t8 - a_t7) / (a_t8 + a_t7)).reshape(1,-1)
    feature = np.concatenate([feature, alpha_temporal], axis=1)

    print(f'alpha_temportal {alpha_temporal}')

    # alpha_asymmetry
    alpha_asymmetry = alpha_frontal + alpha_temporal
    feature = np.concatenate([feature, alpha_asymmetry], axis=1)

    print(f'alpha_asymmetry {alpha_asymmetry}')

    # beta_frontal
    beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
    feature = np.concatenate([feature, beta_frontal], axis=1)

    print(f'beta_frontal {beta_frontal}')


    # beta_temporal
    beta_temporal = ((b_t8 - b_t7) / (b_t8 + b_t7)).reshape(1,-1)
    feature = np.concatenate([feature, beta_temporal], axis=1)

    print(f'beta_temporal {beta_temporal}')

    # print(slow/gamma)
    # print(feature.shape)
    # print(feature)
    # info['all'] = feature
    #print(feature.shape)
    if(type(features) == type(None)): features = feature
    else: features = np.concatenate([features, feature], axis=0)

print(features.shape)
print('=' * 100)

name: exp0
type feature == None band name = Delta value = [[6.95546883]]
band name = Theta value = [[-0.21133261]]


<decorator-gen-201>:4: FutureWarning: NOTE: psd_welch() is a deprecated function. Function ``psd_welch()`` is deprecated; for Raw/Epochs/Evoked instances use ``spectrum = instance.compute_psd(method="welch")`` instead, followed by ``spectrum.get_data(return_freqs=True)``.


band name = Alpha value = [[-4.06205445]]
band name = Beta value = [[-8.69747625]]
band name = Gamma value = [[-13.14332126]]
band name = Slow value = [[-2.73904411]]
band name = Low_beta value = [[-7.48765161]]
relative gamma [[0.20839817]]
alpha_frontal [[-0.02105562]]
alpha_temportal [[0.01229723]]
alpha_asymmetry [[-0.00875839]]
beta_frontal [[-0.03354595]]
beta_temporal [[0.10063094]]
name: exp1
type feature == None band name = Delta value = [[13.44494836]]
band name = Theta value = [[6.57395901]]
band name = Alpha value = [[2.8672477]]
band name = Beta value = [[-1.01074317]]
band name = Gamma value = [[-2.17283104]]
band name = Slow value = [[4.17337843]]
band name = Low_beta value = [[0.01622016]]
relative gamma [[-1.92071006]]
alpha_frontal [[-0.90385039]]
alpha_temportal [[0.3600322]]
alpha_asymmetry [[-0.54381819]]
beta_frontal [[-2.95194804]]
beta_temporal [[-0.09931333]]
name: exp2
type feature == None band name = Delta value = [[9.02933255]]
band name = Theta value = [[1.

In [35]:
feature_column = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta', 'Relative_gamma', 'Alpha_frontal', 'Alpha_temporal',
'Alpha_asymmetry', 'Beta_Frontal', 'Beta_temporal'] #'label'

In [37]:
y_path = './label_exp.csv'
df = pd.read_csv(y_path)
y = np.array(df['label'])
y.shape

(17,)

In [39]:
features.shape, y.shape

((17, 13), (17,))

In [40]:
df_asy = pd.DataFrame(features, columns=feature_column)
df_asy.head()

,Delta,Theta,Alpha,Beta,Gamma,Slow,Low_beta,Relative_gamma,Alpha_frontal,Alpha_temporal,Alpha_asymmetry,Beta_Frontal,Beta_temporal
0,6.955469,-0.211333,-4.062054,-8.697476,-13.143321,-2.739044,-7.487652,0.208398,-0.021056,0.012297,-0.008758,-0.033546,0.100631
1,13.444948,6.573959,2.867248,-1.010743,-2.172831,4.173378,0.016220,-1.920710,-0.903850,0.360032,-0.543818,-2.951948,-0.099313
2,9.029333,1.548254,6.900073,-4.632303,-10.332878,4.243176,-2.389733,-0.410648,-0.031362,0.048796,0.017434,-0.026675,-0.815848
3,11.350209,7.392447,6.185955,0.347076,-1.150481,6.241433,1.088514,-5.425066,0.171033,-0.122356,0.048678,0.695626,-0.239882
4,21.824789,14.998787,8.750110,3.305604,-1.564834,11.077218,5.997419,-7.078845,-0.574923,-0.133568,-0.708491,-0.875918,0.472191


In [53]:
df_asy_alpha = df_asy[['Alpha','Alpha_frontal','Alpha_temporal','Alpha_asymmetry']]
df_asy_gamma = df_asy[['Gamma','Relative_gamma']]
df_asy_beta = df_asy[['Beta','Low_beta','Beta_Frontal','Beta_temporal']]

df_asy_alpha_beta = df_asy[['Alpha','Beta','Low_beta','Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Beta_Frontal','Beta_temporal']]
df_asy_alpha_gamma = df_asy[['Alpha','Beta','Low_beta','Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Gamma','Relative_gamma']]
df_asy_beta_gamma = df_asy[['Beta','Low_beta','Beta_Frontal','Beta_temporal','Gamma','Relative_gamma']]

df_asy_alpha_beta_gamma = df_asy[['Alpha','Beta','Low_beta','Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Beta_Frontal','Beta_temporal','Gamma','Relative_gamma']]

In [54]:
# df_asy_alpha = df_asy[['Alpha_frontal','Alpha_temporal','Alpha_asymmetry']]
# df_asy_gamma = df_asy[['Relative_gamma']]
# df_asy_beta = df_asy[['Beta_Frontal','Beta_temporal']]

# df_asy_alpha_beta = df_asy[['Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Beta_Frontal','Beta_temporal']]
# df_asy_alpha_gamma = df_asy[['Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Relative_gamma']]
# df_asy_beta_gamma = df_asy[['Beta_Frontal','Beta_temporal','Relative_gamma']]

# df_asy_alpha_beta_gamma = df_asy[['Alpha_frontal','Alpha_temporal','Alpha_asymmetry','Beta_Frontal','Beta_temporal','Relative_gamma']]

In [55]:
df_list = [
    df_asy_alpha,
    df_asy_gamma,
    df_asy_beta,
    df_asy_beta_gamma,
    df_asy_alpha_beta,
    df_asy_alpha_gamma,
    df_asy_alpha_beta_gamma,
    df_asy
] # df_asy.iloc[:,7:]

#### Cross Validation

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
lor = LogisticRegression(max_iter=1000)
rfc = RandomForestClassifier(n_jobs=-1)  
svm = SVC(kernel='rbf')
gn = GaussianNB()
knn = KNeighborsClassifier(n_neighbors = 2)
mlp = MLPClassifier()
models = [svm, gn, knn, lor, rfc]
name_list = ['alpha','gamma','beta','beta gamma','alpha beta','alpha gamma','alpha beta gamma','all']
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
print('-----------------SVM-----NB-----KNN----LR----RF-----')
i = 0
for df_each in df_list:
    X_train, X_test, y_train, y_test = train_test_split(df_each, y, test_size=0.2, random_state=999)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    scores = [] 
    for model in models:
        scores.append(
            cross_val_score(model, X_train, np.array(y_train).reshape(-1), cv=5, 
                            scoring='accuracy')
            )
    print(f'{name_list[i]:16} {np.round(np.mean(scores, axis=1),4)}')
    i = i + 1

-----------------SVM-----NB-----KNN----LR----RF-----
alpha            [0.6]
gamma            [0.5333]
beta             [0.5333]
beta gamma       [0.6]
alpha beta       [0.5333]
alpha gamma      [0.5333]
alpha beta gamma [0.5333]
all              [0.5333]


In [43]:
# # split X and y into training and testing sets
# from sklearn.model_selection import train_test_split
# #X_train, X_test, y_train, y_test = train_test_split(df_asy, y[:-1], test_size=0.3, random_state=999)
# X_train, X_test, y_train, y_test = train_test_split(df_psd, y[:-1], test_size=0.3, random_state=999)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [72]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score 
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import pandas as pd

### Training

In [47]:
param_grid = {
    'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=999)
estimator = LogisticRegression(max_iter=1000)
grid = GridSearchCV(estimator  = estimator,
                    param_grid = param_grid,
                    cv = cv,  
                    n_jobs  = -1,
                    refit   = True, 
                    scoring = 'accuracy',
                    return_train_score=True)
                    
grid.fit(X_train, y_train)
print('Accuracy GridSearchCV',grid.best_score_)

Accuracy GridSearchCV 0.7333333333333332


### Testing

In [48]:
from sklearn.metrics import accuracy_score  
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
y_pred = grid.predict(X_test)
target_names = ['without Stress', 'with Stress']
acc = accuracy_score(y_test,y_pred)
crp = classification_report(y_test,y_pred, target_names=target_names)
print("Accuracy :",acc)
print("classification_report : \n",crp)
# print(sns.heatmap(confusion_matrix(y_test,y_pred),annot=True))
# plt.show()

Accuracy : 0.75
classification_report : 
                 precision    recall  f1-score   support

without Stress       0.67      1.00      0.80         2
   with Stress       1.00      0.50      0.67         2

      accuracy                           0.75         4
     macro avg       0.83      0.75      0.73         4
  weighted avg       0.83      0.75      0.73         4

